In [1]:
%load_ext autoreload
%autoreload 2
%pylab inline
pylab.rcParams['figure.figsize'] = (12, 8)
import warnings; warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
from sympy import symbols

import sys; sys.path.append('../')
from PDE_FIND import *

import sys; sys.path.append('../../parametric-discovery/')
from best_subset import *
from frols import frols
from p_linear_regression import PLinearRegression
from r_pca import R_pca
from pde_diff_new import RPCA
from RobustPCA.rpca import RobustPCA

import scipy.io as sio
from scipy.signal import savgol_filter
import itertools

from sklearn.preprocessing import normalize
from sklearn.linear_model import Ridge, BayesianRidge, LinearRegression as SkLinearRegression
from sklearn.base import BaseEstimator
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error
from abess.linear import LinearRegression
import pysindy as ps

from tqdm import trange
from os.path import join as join_path
FIGURE_EXPORT_PATH = "/Users/pongpisit/Documents/figure_export/"

np.random.seed(100)

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib
Sklearn's version: 1.2.2
mrmr is not installed in the env you are using. This may cause an error in future if you try to use the (missing) lib.


In [2]:
def check_percent(bic_scores, complexities):
    slope = (bic_scores[1:]-bic_scores[:-1])/(complexities[1:]-complexities[:-1])
    slope_index = np.argmin(slope)
    percent_improve = 100*np.abs(bic_scores[slope_index+1]-bic_scores[slope_index])/np.abs(bic_scores[slope_index])
    percent_from_1 = 100*np.abs(bic_scores[slope_index+1]-bic_scores[0])/np.abs(bic_scores[0])
    return slope_index+1, percent_improve, percent_from_1

def percent_coeff(pred, ground):
    return 100*(np.abs(pred-ground)/np.abs(ground))

In [3]:
n = 128
data = sio.loadmat(f"../Datasets/reaction_diffusion_3d_{n}.mat")
u_sol = real(data['usol'])
v_sol = real(data['vsol'])
u = np.stack([u_sol, v_sol], axis=-1)
x = real(data['x'][0])
y = real(data['y'][0])
z = real(data['z'][0])
t = real(data['t'][0])

del data

dt = t[1] - t[0]
dx = x[1] - x[0]
dy = y[1] - y[0]
dz = z[1] - z[0]
# For WeakPDELibrary
ground_indices_u = (0, 1, 5, 9, 15, 23)
ground_coeff_u = np.array([0.014, -0.014, -1.000, 0.020, 0.020, 0.020])
ground_indices_v = (2, 5, 10, 16, 24)
ground_coeff_v = np.array([-0.067, 1.0, 0.01, 0.01, 0.01])

x, y, z, t

(array([-1.25      , -1.23046875, -1.2109375 , -1.19140625, -1.171875  ,
        -1.15234375, -1.1328125 , -1.11328125, -1.09375   , -1.07421875,
        -1.0546875 , -1.03515625, -1.015625  , -0.99609375, -0.9765625 ,
        -0.95703125, -0.9375    , -0.91796875, -0.8984375 , -0.87890625,
        -0.859375  , -0.83984375, -0.8203125 , -0.80078125, -0.78125   ,
        -0.76171875, -0.7421875 , -0.72265625, -0.703125  , -0.68359375,
        -0.6640625 , -0.64453125, -0.625     , -0.60546875, -0.5859375 ,
        -0.56640625, -0.546875  , -0.52734375, -0.5078125 , -0.48828125,
        -0.46875   , -0.44921875, -0.4296875 , -0.41015625, -0.390625  ,
        -0.37109375, -0.3515625 , -0.33203125, -0.3125    , -0.29296875,
        -0.2734375 , -0.25390625, -0.234375  , -0.21484375, -0.1953125 ,
        -0.17578125, -0.15625   , -0.13671875, -0.1171875 , -0.09765625,
        -0.078125  , -0.05859375, -0.0390625 , -0.01953125,  0.        ,
         0.01953125,  0.0390625 ,  0.05859375,  0.0

In [4]:
import numpy as np
from scipy.signal import savgol_filter
from scipy.misc import derivative
differentiate = np.gradient

import sgolay2
from savitzkygolay import filter1D, filter2D, filter3D
from function_collections import msemse

In [5]:
u_noisy = np.load("u_noise1e-1.npy")
print(msemse(u_noisy, u))
un = u_noisy[:, :, :, :, 0]
vn = u_noisy[:, :, :, :, 1]
del u_noisy

6.281464129182443e-08


In [6]:
# und = np.stack([filter3D(un[:, :, :, i], mu=5, poly=3, order=0) for i in trange(un.shape[-1])], axis=-1)
# vnd = np.stack([filter3D(vn[:, :, :, i], mu=5, poly=3, order=0) for i in trange(vn.shape[-1])], axis=-1)
# u_noisy = np.stack([und, vnd], axis=-1)
# msemse(u_noisy, u)

In [7]:
div = 30
ws = max(un.shape[:-2])//div; po = 5
if ws%2 == 0: ws -=1
ws

nun = np.zeros_like(un)
for i in trange(un.shape[-1]):
    for j in range(un.shape[-2]):
        nun[:, :, j, i] = sgolay2.SGolayFilter2(window_size=ws, poly_order=po)(un[:, :, j, i])
un = nun.copy()
del nun
nvn = np.zeros_like(vn)
for i in trange(vn.shape[-1]):
    for j in range(vn.shape[-2]):
        nvn[:, :, j, i] = sgolay2.SGolayFilter2(window_size=ws, poly_order=po)(vn[:, :, j, i])
vn = nvn.copy()
del nvn

msemse(np.stack([un, vn], axis=-1), u)

100%|███████████████████████████████████████████████████████████████████████| 100/100 [00:14<00:00,  6.70it/s]


6.281464129182435e-08

In [8]:
dim = 10

un = un.reshape(-1, len(t))
uun, sigmaun, vun = np.linalg.svd(un, full_matrices=False); vun = vun.T
un = uun[:,0: dim].dot(np.diag(sigmaun[0:dim]).dot(vun[:,0:dim].T))
un = un.reshape(len(x), len(y), len(z), len(t))

vn = vn.reshape(-1, len(t))
uvn, sigmavn, vvn = np.linalg.svd(vn, full_matrices=False); vvn = vvn.T
vn = uvn[:,0: dim].dot(np.diag(sigmavn[0:dim]).dot(vvn[:,0:dim].T))
vn = vn.reshape(len(x), len(y), len(z), len(t))

u_noisy = np.stack([un, vn], axis=-1)

In [ ]:
# np.save("u_noise1e-1_denoised.npy", u_noisy)